In [ ]:
from BacktestZurichStrategy import BacktestZurichStrategy
import backtrader as bt
import warnings
import yfinance as yf
import datetime as dt

'''This source code is for Single Objective optimization, for each experiment, all you need is to change the objective function returned by obj_fun(x)'''

warnings.filterwarnings("ignore")

from pyswarm import pso 
import pyswarms as ps
# The objective function for optimization
def obj_fun(x):
    print('')
    print('Launched the iteration with ' + str(x))

    ap = {'bbperiod': int(x[0]),
          'nbdevup': float(x[1]),
          'nbdevdn': float(x[2]),
          
          'fastvol_period' : int(x[3]),
          'slowvol_period' : int(x[4]),
          
           'fastbbw_period': int(x[5]),
            'slowbbw_period': int(x[6]),
            
            'veryfastsma_period': int(x[7]),
            'fastsma_period': int(x[8]),
            'midsma_period': int(x[9]),
            'slowsma_period': int(x[10]),
            'veryslowsma_period' : int(x[11]),

            'vlifast_period': int(x[12]),
            'vlislow_period': int(x[13]),
            'vlitop_period' : int(x[14]),

                    
            'dynamic_sl_long_bull': x[15],
            'dynamic_sl_long_bear': x[16],

            'dynamic_sl_short_bull': x[17],
            'dynamic_sl_short_bear': x[18]
        
          
          }
    os = {'order_full': False,
          'order_status': False,
          'trades': False,
          'performance': True,
          'plot': False
          }

    # Creater object for Backtesting

    # include the training data
    data = './data/data_NEOUSDT_1h_train.csv'
    backtest = BacktestZurichStrategy(file_data=data,
                                      algo_params=ap,
                                      output_settings=os)
    # Run the strategy (hourly timeframe)
    backtest.run_strategy(cash=100000,
                          commission=0.001,
                         tf = bt.TimeFrame.Minutes, 
                         compression = 60)
    
   
    # Add "minus" for minimization
    # These are the objective functions to be optimized(4)
    
    sharpe = -backtest.sharpe_ratio
    calmar = -backtest.calmar_ratio
    stability = -backtest.stability 
    sortino = -backtest.sortino_ratio
    return sortino

# Bounds for parameters space


lb = [20, 2,2,
       10,50 ,
          10,50,
          10,20,50, 100,200,
          200, 1000,1000,
        0.80,0.80,
        1.00,1.00]
ub = [21,2.1,2.1,
      11, 51,
      11,51,
        11,21,51, 101,201,
        201,1001,1001,
        0.99,0.99,
        1.20,1.20]

# Run the optimization with 500 swarms for 1000 iterations
print('before optimization')
xopt, fopt = pso(obj_fun, lb, ub, swarmsize=500, maxiter=1000)
print('after optimization')
print('OPTIMAL PARAMETERS:')
print(xopt, fopt)



In [ ]:
from pymoo.algorithms.soo.nonconvex.pso import PSO

from pymoo.optimize import minimize

import numpy as np
from pymoo.core.problem import ElementwiseProblem
from BacktestZurichStrategy import BacktestZurichStrategy
import backtrader as bt
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

from pymoo.algorithms.moo.moead import MOEAD
from pymoo.optimize import minimize
from pymoo.problems import get_problem
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter



'''This code is for Multi_Objective Optimization '''

class problem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var= 19,
                         n_obj=1,
                         n_ieq_constr=0,
                         xl=np.array(  [20, 2,2,
       10,50 ,
          10,50,
          9,27,38, 141,167,
          200, 1000,1000,
        0.85,0.90,
        1.00,1.00]),
                         xu=np.array([21,2.1,2.1,
      11, 51,
      11,51,
        10,28,39, 142,168,
        201,1001,1001,
        0.99,0.99,
        1.10,1.20]))
        
        
    

    def _evaluate(self, x, out, *args, **kwargs):
        def obj_fun(x):
            print('')
            print('Launched the iteration with ' + str(x))

            ap = {'bbperiod': int(x[0]),
                'nbdevup': float(x[1]),
                'nbdevdn': float(x[2]),
                
                'fastvol_period' : int(x[3]),
                'slowvol_period' : int(x[4]),
                
                'fastbbw_period': int(x[5]),
                    'slowbbw_period': int(x[6]),
                    
                    'veryfastsma_period': int(x[7]),
                    'fastsma_period': int(x[8]),
                    'midsma_period': int(x[9]),
                    'slowsma_period': int(x[10]),
                    'veryslowsma_period' : int(x[11]),

                    'vlifast_period': int(x[12]),
                    'vlislow_period': int(x[13]),
                    'vlitop_period' : int(x[14]),

                    
                    
                    'dynamic_sl_long_bull': x[15],
                    'dynamic_sl_long_bear': x[16],

                    'dynamic_sl_short_bull': x[17],
                    'dynamic_sl_short_bear': x[18]
                
                
                }
            os = {'order_full': False,
                'order_status': False,
                'trades': False,
                'performance': True,
                'plot': False
                }

            # Creater object for Backtesting
            
            
            data = './data/data_BTCUSDT_1h_train.csv'
            backtest = BacktestZurichStrategy(file_data=data,
                                            algo_params=ap,
                                            output_settings=os)
            # Run the strategy (hourly timeframe)
            backtest.run_strategy(cash=100000,
                                commission=0.001,
                                tf=bt.TimeFrame.Minutes,
                                compression=60)
            
        
            # Add "minus" for minimization

            sharpe = -backtest.sharpe_ratio
            calmar = -backtest.calmar_ratio
            stability = -backtest.stability 
            sortino = -backtest.sortino_ratio
            return [sortino, calmar]

        
        sortino = obj_fun(x)[0]
        calmar = obj_fun(x)[1]
        

       
# as mentioned in the research article max(sortino)+ max(calmar)
        out["F"] = [sortino, calmar]
        
        


problem = problem()




ref_dirs = get_reference_directions("uniform", 2, n_partitions=12)

algorithm = MOEAD(
    ref_dirs,
    n_neighbors=15,
    prob_neighbor_mating=0.7,
)

res = minimize(problem,
               algorithm,
               ('n_gen', 500),
               seed=1,
               verbose=False)

Scatter().add(res.F).show()


